# GridSearch + Validation croisée + Hyperparamètres (hors pipeline)

In [1]:
## 4.0 Setup + data (Wine)

import numpy as np

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# 1) Dataset réel
X, y = load_wine(return_X_y=True)

# 2) Split: train / test (le test ne sert qu'à la fin)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape, "| Test shape:", X_test.shape)

Train shape: (142, 13) | Test shape: (36, 13)


## 4.1 Paramètres vs Hyperparamètres

**Paramètres (learned / appris)** : appris pendant `.fit()`

* Ex: coefficients `w` d’une régression logistique.

**Hyperparamètres (choisis / réglés)** : choisis avant l’entraînement

* Ex: `n_neighbors` pour KNN, `C` (force de régularisation) pour LogisticRegression.

Mini “preuve” : KNN n’a pas de paramètres appris (il mémorise), mais a des hyperparamètres (k, distance, etc.).


## 4.2 Version pas à pas : un “jeu de validation” (hold-out) pour choisir k

In [2]:
### Étape A — créer un validation set (train/val/test)

X_subtrain, X_val, y_subtrain, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)
# -> 0.25 de train => ~20% total pour val (car X_train est 80% du total)

print("Subtrain:", X_subtrain.shape, "| Val:", X_val.shape, "| Test:", X_test.shape)

Subtrain: (106, 13) | Val: (36, 13) | Test: (36, 13)


In [ ]:
### Étape B — preprocessing manuel (fit scaler sur subtrain uniquement)

scaler = StandardScaler()
X_subtrain_s = scaler.fit_transform(X_subtrain)
X_val_s = scaler.transform(X_val)
X_test_s = scaler.transform(X_test)  # on transformera le test avec le scaler appris sur subtrain

In [4]:
### Étape C — grid search “manuel” sur k (hyperparamètre) en utilisant la validation

k_values = range(1, 31)
val_scores = []

best_k = None
best_val_acc = -1

for k in k_values:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_subtrain_s, y_subtrain)

    y_val_pred = model.predict(X_val_s)
    acc = accuracy_score(y_val, y_val_pred)
    val_scores.append(acc)

    if acc > best_val_acc:
        best_val_acc = acc
        best_k = k

print("Meilleur k (selon val):", best_k, "| Accuracy val:", round(best_val_acc, 3))


Meilleur k (selon val): 7 | Accuracy val: 1.0


In [5]:
### Étape D — évaluation finale sur le test (une seule fois)

final_model = KNeighborsClassifier(n_neighbors=best_k)
final_model.fit(X_subtrain_s, y_subtrain)

y_test_pred = final_model.predict(X_test_s)
print("Accuracy test (final):", round(accuracy_score(y_test, y_test_pred), 3))

Accuracy test (final): 0.972


#### Point clé :

* Le **jeu de validation** sert à choisir l’hyperparamètre
* Le **test** sert à estimer la performance “réelle” finale (1 seule fois)


## 4.3 Validation croisée (CV) + grid search manuel (pas à pas sans pipeline)

- Le hold-out (1 split) dépend beaucoup du hasard.  

- La **validation croisée** fait plusieurs splits, et on moyenne.

In [6]:
## Étape A — définir la CV

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [ ]:


## Étape B — grid search manuel CV **correct** (scaler refit à chaque fold)

# **pourquoi pipeline existe** : sinon tu dois faire ça à la main pour éviter la fuite.


def cv_score_knn_with_scaling(X, y, k, cv):
    fold_scores = []
    for train_idx, val_idx in cv.split(X, y):
        X_tr, X_va = X[train_idx], X[val_idx]
        y_tr, y_va = y[train_idx], y[val_idx]

        scaler = StandardScaler()
        X_tr_s = scaler.fit_transform(X_tr)
        X_va_s = scaler.transform(X_va)

        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X_tr_s, y_tr)

        pred = model.predict(X_va_s)
        fold_scores.append(accuracy_score(y_va, pred))
    return float(np.mean(fold_scores))

k_values = range(1, 31)
cv_scores = []

best_k_cv = None
best_cv_acc = -1

for k in k_values:
    acc = cv_score_knn_with_scaling(X_train, y_train, k, cv)
    cv_scores.append(acc)
    if acc > best_cv_acc:
        best_cv_acc = acc
        best_k_cv = k

print("Meilleur k (CV):", best_k_cv, "| Accuracy CV:", round(best_cv_acc, 3))

In [ ]:
## Étape C — entraîner le modèle final sur tout le train, tester sur test

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

final_knn = KNeighborsClassifier(n_neighbors=best_k_cv)
final_knn.fit(X_train_s, y_train)

print("Accuracy test:", round(final_knn.score(X_test_s, y_test), 3))

Messages clés :

* CV = plus robuste qu’un seul split
* Pour être “clean”, le preprocessing doit être refit **dans chaque fold**
* C’est exactement ce que **Pipeline** automatise

## 4.4 “État de l’art” : `GridSearchCV` sur données déjà transformées


**Important** : si tu scales **une seule fois** avant la CV, tu crées une **fuite** (le scaler a “vu” toute la distribution du train, y compris les folds de validation).  

Donc c’est “pratique”, mais **pas strictement correct** → c’est la passerelle parfaite pour justifier Pipeline ensuite.  

In [ ]:
# Version compacte :

from sklearn.model_selection import GridSearchCV

# Scaling une seule fois (pratique mais leakage pour la CV)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)

param_grid = {"n_neighbors": list(range(1, 31))}
grid = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid,
    cv=5,
    scoring="accuracy"
)

grid.fit(X_train_s, y_train)

print("Best params:", grid.best_params_)
print("Best CV score:", round(grid.best_score_, 3))

# test final
X_test_s = scaler.transform(X_test)
print("Test score:", round(grid.score(X_test_s, y_test), 3))

Conclusion :

* `GridSearchCV` est top
* **Mais** si tu veux une CV propre, il faut que le scaler soit “dans le loop” → donc **Pipeline** (partie suivante)

# 4.5 Bonus (hyperparamètre différent) : Logistic Regression et `C`

Même structure, autre hyperparamètre :

In [ ]:
def cv_score_logreg_with_scaling(X, y, C, cv):
    fold_scores = []
    for train_idx, val_idx in cv.split(X, y):
        X_tr, X_va = X[train_idx], X[val_idx]
        y_tr, y_va = y[train_idx], y[val_idx]

        scaler = StandardScaler()
        X_tr_s = scaler.fit_transform(X_tr)
        X_va_s = scaler.transform(X_va)

        model = LogisticRegression(C=C, max_iter=5000, solver="lbfgs")
        model.fit(X_tr_s, y_tr)

        pred = model.predict(X_va_s)
        fold_scores.append(accuracy_score(y_va, pred))
    return float(np.mean(fold_scores))

C_values = [0.01, 0.1, 1, 10, 100]

best_C = None
best_acc = -1

for C in C_values:
    acc = cv_score_logreg_with_scaling(X_train, y_train, C, cv)
    print("C =", C, "-> CV acc =", round(acc, 3))
    if acc > best_acc:
        best_acc = acc
        best_C = C

print("Meilleur C:", best_C, "| Best CV:", round(best_acc, 3))

## Récapitulatif global des sujets abordés dans cette partie :

* Différence **paramètres vs hyperparamètres**

* Rôle du **jeu de validation**

* Pourquoi le **test set** ne sert pas au tuning

* Pourquoi **CV** est plus robuste

* Pourquoi preprocessing + CV à la main devient pénible → **Pipeline** arrive naturellement